In [10]:
# Text text processing library
import torchtext
from torchtext import data
from torchtext import datasets
from torchtext.vocab import Vectors
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import itertools as it
# from models import *
# from helpers import *
# import main
import matplotlib.pyplot as plt
import spacy
MAX_LEN = 20
MIN_FREQ = 5

In [6]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

BOS_WORD = '<s>'
EOS_WORD = '</s>'
DE = data.Field(tokenize=tokenize_de)

# only target needs BOS/EOS:
EN = data.Field(tokenize=tokenize_en, init_token = BOS_WORD, eos_token = EOS_WORD) 

train, val, test = datasets.IWSLT.splits(exts=('.de', '.en'), fields=(DE, EN), 
                                         filter_pred=lambda x: len(vars(x)['src']) <= MAX_LEN and 
                                         len(vars(x)['trg']) <= MAX_LEN)

In [13]:
DE.build_vocab(train.src, min_freq=MIN_FREQ)
EN.build_vocab(train.trg, min_freq=MIN_FREQ)

train_iter, val_iter = data.BucketIterator.splits((train, val), batch_size=32, device=-1,
                                                  repeat=False, sort_key=lambda x: len(x.src))

In [44]:
batch = next(iter(train_iter))
idx = 1
print("Source")
print(' '.join([DE.vocab.itos[w] for w in batch.src.data[:,idx]]))
print("Target")
print(' '.join([EN.vocab.itos[w] for w in batch.trg.data[:,idx]]))

Source
Ein Stück <unk> an einer Baustelle in Italien .
Target
<s> Just a piece of barrier tape construction stuff in Italy . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


In [45]:
batch.src[:,0:1]

Variable containing:
  120
    3
  365
   67
   20
   29
 2638
  948
    2
[torch.LongTensor of size 9x1]